In [1]:
import pickle
from pathlib import Path
from PIL import Image
import torch
import torchvision.transforms.functional as tvf

torch.set_grad_enabled(False)

from lvae.models.qresvae.zoo import qres34m_lossless

d:\libraries\anaconda3\envs\pt20env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialize model and load pre-trained weights

In [2]:
model = qres34m_lossless(pretrained=True)

model.eval()
model.compress_mode(True)

Downloading: "https://huggingface.co/duanzh0/my-model-weights/resolve/main/qres34m/qres34m-lossless.pt" to C:\Users\duanz/.cache\torch\hub\checkpoints\qres34m-lossless.pt
100%|██████████| 130M/130M [00:03<00:00, 39.3MB/s] 


Compress an RGB image

In [3]:
img_path = '../../images/collie128.png'

im = tvf.to_tensor(Image.open(img_path)).unsqueeze_(0)
compressed_obj = model.compress(im)

Save to file, compute bit rate

In [4]:
save_path = '../../runs/image.bits'
with open(save_path, 'wb') as f:
    pickle.dump(compressed_obj, file=f)

total_bits = Path(save_path).stat().st_size * 8
bpp = total_bits / (im.shape[2] * im.shape[3])
print(f'Compressed file size: {total_bits} bits = {bpp:.6f} bpp')

Compressed file size: 286112 bits = 17.462891 bpp


Decompress and reconstruct the image

In [5]:
with open(save_path,'rb') as f:
    compressed_obj = pickle.load(file=f)

im_hat = model.decompress(compressed_obj).squeeze(0).cpu()

Check if the compression is lossless

In [9]:
real = tvf.pil_to_tensor(Image.open(img_path)) # uint8
fake = torch.round_(im_hat * 255.0).to(dtype=torch.uint8)

torch.equal(real, fake)

True